# Data processing

The water monitoring project at the Montreux Jazz has been going on since 2016. The data has been collected and treated by a variety of people since then. 

__Objective:__ Standardize the nomenclature from the different sampling years. Provide a model for storing and collecting data in the future.

__Purpose:__ Define the probability that a survey will exceed a threshold value within the period of the year defined by the survey results.

## Definitions

* colony: a circular discoloration of the media within a defined size and color range
* colony-count: the number of discolorations of the same hue for a media type
* media/medium: the environment that the water samples are placed in
* color: the observed color of the colony
* label: the assumed category of the color:
  * Bioindicator
  * Coliform
  * Other
* coef:  the correction factor applied, to allow reporting of colony counts per 100ml of the original water sample.
 
The purpose of the sampling is to identify colonies that appear in the media and classify them as one of the possible labels. The label of interest is _Bioindicators_, this represents the bacteria that are issue from the organism of interest. The organism in this case is people, the _Bioindicator_ is issue from fecal contaminants.

## Methods

The process requires collaborating with the data-manager(s) from the different project years and ensuring that the data from each year can be combined and interpreted together. The data for this collaboration is stored in the _componentdata_ folder.

The relationship of previous label <---> new label is stored in a dictionary or an array for the different possibilities of medium, color, label and coefficient. The new labels are applied to a data-frame.

The finsihed data (the result of the collaboration) is stored in the _end_ folder

## Sample data

The sample data is an example of the desired output per year. This includes the following parameters:

1. colony-count
2. label
3. location
4. coeficient*count
5. week number
6. day of year
7. is-jazz: boolean
8. rain fall in millimeters

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

project = "Hackuarium do it together water quality sampling"
site_markers = {"SVT":"o", "VNX":"D", "MRD":"X"}
species_colors = { "Bioindicator":"dodgerblue", "Coliform":"magenta"}
marker_colors = {"SVT":"black", "VNX":"green", "MRD":"goldenrod"}
sites = ["SVT", "VNX", "MRD"]

## Survey data

The format of the survey data prior to processing. The result of the collaboration.

In [2]:
df = pd.read_csv("data/end/survey_data_2020_2023.csv")
df['date'] = pd.to_datetime(df["date"])
# df['week'] = df["date"].dt.isocalendar().week
# df['isjazz'] = False
df.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,isjazz,label,medium
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A


### Applying labels

The colors that were used for the observations can be placed into three broad categories. 

1. Bioindicator
2. Coliforms
3. Other

The microbiologist determines the correct label for the recorded color based on the specifics of the media/medium used to grow the culture.

The colors appropriate to each label are stored in an array. The color for each record is tested for membership in one of the arrays. If it is in one of the arrays, the name of that array is returned. If the color is not in any array the original value is returned. The result is added to the data-frame.

```python
bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met"]
coliforms = ["Pink", "pink"]
other = ["Turquoise", "Turquoise slow", "other"]

def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

df["label"] = df.color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))
```

We do the same for the media/medium except we use a dictionary to store that information

```python
media_names = {
    "ECC-A Card":"ECC-A",
    "new ECCA":"ECC-A",
    "E-coli side": "E coli",
    "ECC-side":"ECC",
    "selective":"Levine",
    "media":"EasyGel",
    "plus uv":"EasyGelPlus",
    "UVplus":"EasyGelPlus",
    "non-restrictive":"LB"
}

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x


df["medium"] = df.media.apply(lambda x: translate_media(x, media_names))

```



### Labeling the date range of interest

Voici les dates de Jazz pour toutes les années de prélèvement :

* 2016:  2016-07-01 - 2016-07-16
* 2017: 2017-06-30 - 2017-06-15
* 2020: 2020-07-03 - 2020-07-18
* 2022: 2022-07-01 - 2022-07-16
* 2023: 2023-06-30 - 2023-07-15

In [3]:
# mask the date ranges
import datetime as dt

def make_date_object(x):
    return dt.datetime.strptime(x, "%Y-%m-%d")

# y1 = (df['date'] >= make_date_object("2020-07-03" )) & (df['date'] <= make_date_object("2020-07-18"))
# y2 = (df['date'] >= make_date_object("2022-07-01" )) & (df['date'] <= make_date_object("2022-07-16"))
# y3 = (df['date'] >= make_date_object("2023-06-30" )) & (df['date'] <= make_date_object("2023-07-15"))

# df.loc[y1, "isjazz"] = True
# df.loc[y2, "isjazz"] = True
# df.loc[y3, "isjazz"] = True
df.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,isjazz,label,medium
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A


In [4]:
# translate colors
def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met"]
coliforms = ["Pink", "pink", "purple"]
other = ["Turquoise", "Turquoise slow", "other", "mauve"]

# df["label"] = df.color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x

media_names =  {
"ECC-A Card":"ECC-A",
"new ECCA":"ECC-A",
"E-coli side": "E coli",
"Double side E coli": "E coli",
"ECC-side":"ECC",
"Double side ECC":"ECC",
"selective":"Levine",
"media":"EasyGel",
"plus uv":"EasyGelPlus",
"UVplus":"EasyGelPlus",
"non-restrictive":"LB",
"levine": "Levine"
}

# df["medium"] = df.media.apply(lambda x: translate_media(x, media_names))

In [5]:
df["medium"].unique()

array(['ECC-A', 'ECC', 'E coli', 'LB', 'Levine'], dtype=object)

In [6]:
df["label"].unique()

array(['Bioindicator', 'Other', 'Coliform'], dtype=object)

In [7]:
df.coef.unique()

array([100.,   1.])

In [8]:
df.year.unique()

array([2023, 2022, 2020])

In [9]:
df.week.unique()

array([24, 25, 26, 27, 28, 29, 30, 31])

In [10]:
df = df[df.location.isin(sites)]
df["count"] = df["count"].astype('int', errors='ignore')
df[df["count"] == 'nd']

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,isjazz,label,medium
1105,2022-07-18,VNX2,NaN,ECC,Dark Blue,nd,nd,100.0,"('18/7/2022', 'VNX2')",2022,VNX,199,29,False,Bioindicator,ECC
1106,2022-07-18,VNX2,NaN,ECC,Turquoise,nd,nd,100.0,"('18/7/2022', 'VNX2')",2022,VNX,199,29,False,Other,ECC
1107,2022-07-18,VNX2,NaN,ECC,Pink,nd,nd,100.0,"('18/7/2022', 'VNX2')",2022,VNX,199,29,False,Coliform,ECC
1108,2022-07-18,VNX3,NaN,ECC,Dark Blue,nd,nd,100.0,"('18/7/2022', 'VNX3')",2022,VNX,199,29,False,Bioindicator,ECC
1109,2022-07-18,VNX3,NaN,ECC,Turquoise,nd,nd,100.0,"('18/7/2022', 'VNX3')",2022,VNX,199,29,False,Other,ECC
1110,2022-07-18,VNX3,NaN,ECC,Pink,nd,nd,100.0,"('18/7/2022', 'VNX3')",2022,VNX,199,29,False,Coliform,ECC
1117,2022-07-18,VNX6,NaN,ECC,Dark Blue,nd,nd,100.0,"('18/7/2022', 'VNX6')",2022,VNX,199,29,False,Bioindicator,ECC
1118,2022-07-18,VNX6,NaN,ECC,Turquoise,nd,nd,100.0,"('18/7/2022', 'VNX6')",2022,VNX,199,29,False,Other,ECC
1119,2022-07-18,VNX6,NaN,ECC,Pink,nd,nd,100.0,"('18/7/2022', 'VNX6')",2022,VNX,199,29,False,Coliform,ECC
